In [ ]:
from google.colab import drive
drive.mount('/content/itsdrive')

Mounted at /content/itsdrive


In [ ]:
import imgaug as ia
from imgaug import augmenters as iaa
import numpy as np
import imageio
from PIL import Image

# Create a sequence of augmentations with relevant augmentations
seq = iaa.Sequential([
    iaa.Affine(translate_percent={"x": (-0.25, 0.25), "y": (-0.4, 0.4)}),  # Translation with smaller range
    iaa.Fliplr(0.5),  # Horizontal flipping
])

# Load an image
image = Image.open("/content/itsdrive/MyDrive/Aurora/2D vectors/Screenshot from 2023-10-26 20-12-21 (copy).png")
image = image.convert("RGB")  # Convert to RGB mode

# Define the relative bounding box coordinates (center_x, center_y, width, height)
center_x, center_y, relative_width, relative_height = 0.433196, 0.475424, 0.328329, 0.125182

# Calculate absolute bounding box coordinates based on relative values
image_width, image_height = image.size
x1 = int((center_x - relative_width / 2) * image_width)
y1 = int((center_y - relative_height / 2) * image_height)
x2 = int((center_x + relative_width / 2) * image_width)
y2 = int((center_y + relative_height / 2) * image_height)

# Create a BoundingBox object
bounding_box = ia.BoundingBox(x1=x1, y1=y1, x2=x2, y2=y2)

# Create a BoundingBox object list (only one in this example)
bounding_boxes = [bounding_box]

# Define a counter for naming images
counter = 0

# Specify the directory to save the augmented images and labels
save_directory = "/content/itsdrive/MyDrive/Aurora/Yolov8/data/images/train/"
labels_directory = "/content/itsdrive/MyDrive/Aurora/Yolov8/data/labels/train/"

# Apply augmentations and save images and labels
for _ in range(100):  # Generate 1000 augmented images (adjust as needed)
    image_aug, bounding_boxes_aug = seq(image=np.array(image), bounding_boxes=bounding_boxes)
    bounding_boxes_aug = [bb.clip_out_of_image(np.array(image)) for bb in bounding_boxes_aug]

    # Convert the augmented image back to PIL Image
    image_aug = Image.fromarray(image_aug)

    # Save the augmented image with a unique filename based on the counter
    image_filename = f"{save_directory}augmented_image_{counter}.jpg"
    image_aug.save(image_filename)

    # Save the corresponding label in a text file
    label_filename = f"{labels_directory}augmented_image_{counter}.txt"
    with open(label_filename, 'w') as label_file:
        # Write the bounding box coordinates in the specified format
        x1, y1, x2, y2 = bounding_boxes_aug[0].x1, bounding_boxes_aug[0].y1, bounding_boxes_aug[0].x2, bounding_boxes_aug[0].y2
        width = x2 - x1
        height = y2 - y1
        center_x = (x1 + x2) / 2 / image_width
        center_y = (y1 + y2) / 2 / image_height
        label_file.write(f"0 {center_x:.6f} {center_y:.6f} {width / image_width:.6f} {height / image_height:.6f}")

    # Increment the counter
    counter += 1


KeyboardInterrupt: ignored

In [ ]:
!pip install ultralytics
from ultralytics import YOLO

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.8/644.8 kB 13.2 MB/s eta 0:00:00


In [ ]:
model=YOLO("yolov8n.yaml")


                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128

In [ ]:
results=model.train(data="/content/itsdrive/MyDrive/Aurora/Yolov8/mag.yaml",epochs=100)

Ultralytics YOLOv8.0.203 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=/content/itsdrive/MyDrive/Aurora/Yolov8/mag.yaml, epochs=100, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torch

In [ ]:
import os
from ultralytics import YOLO
import cv2

# Directory where your images are /contestored
IMAGES_DIR = '/content/itsdrive/MyDrive/Aurora/ggg'
OUTPUT_DIR = '/content/itsdrive/MyDrive/Aurora/Yolov8/predictions'

# List of image files in the directory
image_files = [os.path.join(IMAGES_DIR, filename) for filename in os.listdir(IMAGES_DIR) if filename.endswith(('.jpg', '.png'))]

model_path = os.path.join('.', 'runs', 'detect', 'train', 'weights', 'last.pt')

# Load a model
model = YOLO('runs/detect/train/weights/best.pt')  # load a custom model

threshold = 0.3

for image_file in image_files:
    # Load the image
    frame = cv2.imread(image_file)
    H, W, _ = frame.shape

    results = model(frame)[0]

    for result in results.boxes.data.tolist():
        x1, y1, x2, y2, score, class_id = result

        if score > threshold:
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 4)
            cv2.putText(frame, results.names[int(class_id)].upper(), (int(x1), int(y1 - 10)),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 255, 0), 3, cv2.LINE_AA)

    # Save the annotated image
    output_file = os.path.join(OUTPUT_DIR, os.path.basename(image_file))
    cv2.imwrite(output_file, frame)


FileNotFoundError: ignored

In [ ]:
import os
from ultralytics import YOLO
import cv2

# Load the YOLO model
model = YOLO('runs/detect/train/weights/best.pt')

# Create a VideoCapture object to capture live video from the front camera (use 0 for the default camera)
cap = cv2.VideoCapture(0)

# Set the frame dimensions (adjust as needed)
cap.set(3, 1280)  # Width
cap.set(4, 720)   # Height

# Create a window to display the live video
cv2.namedWindow('Live Object Detection', cv2.WINDOW_NORMAL)

threshold = 0.5

while True:
    ret, frame = cap.read()

    # Perform object detection on the frame
    results = model(frame)

    for result in results.boxes.data.tolist():
        x1, y1, x2, y2, score, class_id = result

        if score > threshold:
            label = f'{results.names[int(class_id)].upper()}: {score:.2f}'
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 4)
            cv2.putText(frame, label, (int(x1), int(y1 - 10)),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 255, 0), 3, cv2.LINE_AA)

    # Display the annotated frame
    cv2.imshow('Live Object Detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the VideoCapture and close the window
cap.release()
cv2.destroyAllWindows()


In [ ]:
import os
from ultralytics import YOLO
import cv2
from google.colab import files
from IPython.display import HTML, display
from IPython.display import clear_output  # Add this import

# Load the YOLO model
model = YOLO('runs/detect/train/weights/best.pt')

# Upload the video file
uploaded_files = files.upload()

# Assume that the uploaded video file is named 'video.mp4'
video_file = 'video.mp4'

# Create a VideoCapture object to read the uploaded video
cap = cv2.VideoCapture(video_file)

# Get the frames per second (fps) of the video
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Create an output VideoWriter to save the annotated video
output_file = 'output_video.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_file, fourcc, fps, (1920, 1080))

threshold = 0.5

while True:
    ret, frame = cap.read()

    if not ret:
        break

    # Perform object detection on the frame
    results = model(frame)

    for result in results.boxes.data.tolist():
        x1, y1, x2, y2, score, class_id = result

        if score > threshold:
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 4)
            label = f'{results.names[int(class_id)].upper()}: {score:.2f}'
            cv2.putText(frame, label, (int(x1), int(y1 - 10)),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 255, 0), 3, cv2.LINE_AA)

    # Write the annotated frame to the output video
    out.write(frame)

# Release the VideoCapture and VideoWriter objects
cap.release()
out.release()

# Display the video directly in Colab using HTML
video_path = f'/content/{output_file}'
HTML(f'<video width="800" height="600" controls><source src="{video_path}" type="video/mp4"></video>')

# If you encounter a blank output, try running the cell again or clearing the output using clear_output()
# clear_output(wait=True)


Saving VID20231030223923.mp4 to VID20231030223923 (3).mp4


In [ ]:
# Display the video directly in Colab using HTML
video_path = f'/content/{output_file}'
HTML(f'<video width="800" height="600" controls><source src="{video_path}" type="video/mp4"></video>')

In [ ]:
out_link = f'<a href="{output_file}" download>Download Annotated Video</a>'
display(HTML(out_link))


In [ ]:

# Play the annotated video directly in Colab
display(HTML(f'<video controls autoplay src="{output_file}" width="800" height="600"></video>'))

In [ ]:
import cv2

# Load the image
image = cv2.imread('/content/itsdrive/MyDrive/Aurora/2D vectors/Screenshot from 2023-10-26 20-12-21 (copy).png')

# Get the height and width of the image
height, width, _ = image.shape

# Print the image size
print(f"Image width: {width} pixels")
print(f"Image height: {height} pixels")
